# Example using pellet sorter

This notebook contains an example of how to run the code for using the multimaterial dispenser. Most cells can be run without any modification if the dispenser has been built without design modifications. The lines that need modifications are marked like this:

`###############################################################################`

In [1]:
# Import Libraries
import serial
import numpy as np
import time
import serial.tools.list_ports
import sys
sys.path.append('src')
from DispenserSetup import SetupDispenser, Compositions

In this cell, we define the port name for the arduino and the balance. We do it by listing the connected ports and selecting the ones whose device serial is the corresponding to our device. You should find the serial of your devices and change it accordingly. You can do it by inspecting the devices detected by serial.tools.list_ports.comports(). In case of doubt, try disconnecting and connecting to see which ports are affected.

In [2]:
ports = serial.tools.list_ports.comports()

###############################################################################

arduino_serial = "write_here_your_arduino_serial"
balance_serial = "write_here_your_balance_serial"

###############################################################################

for port in ports:
    if port.serial_number == arduino_serial:
        arduino_port = port.device
    elif port.serial_number == balance_serial:
        balance_port = port.device
        
### Serial initialization
## Arduino
arduino = serial.Serial(arduino_port, 9600)
## Balance
balance = serial.Serial(port=balance_port, baudrate=9600, bytesize=serial.SEVENBITS,
                       parity=serial.PARITY_EVEN, stopbits=serial.STOPBITS_ONE)

NameError: name 'arduino_port' is not defined

The next step is to define the materials and their concentrations in each of the dispensers. For that we will use the `SetupDispenser` function from `/src/DispenserSetup`. It will require a `.csv` file detailing the compositions, that can be built following the example available in the help of the function:

In [3]:
help(SetupDispenser)

Help on function SetupDispenser in module DispenserSetup:

SetupDispenser(dispenser_setup_file, Polymer)
    Funtion to read the setup file and create structures to define what material each dispenser has.
    
    Inputs:
    
        dispenser_setup_file: full route of a .csv file with the different materials and how they are arranged. Example:
            Dispenser, PLA, GNP, CLO, BN
            1, 0.0, 0.0, 0.0, 0.0 # Empty
            2, 0.0, 0.0, 0.0, 0.0 # Empty
            3, 0.0, 0.0, 0.0, 0.0 # Empty
            4, 1.0, 0.0, 0.0, 0.0 # Contains pure polymer
            5, 0.9, 0.1, 0.0, 0.0 # Polymer with 10% GNP
            6, 0.9, 0.0, 0.0, 0.1 # Polymer with 10% BN
            7, 0.0, 0.0, 0.0, 0.0 # Empty
            8, 0.0, 0.0, 0.0, 0.0 # Empty
            
            Notes:
            - Only one polymer matrix allowed.
            - All of the non-empty dispensers must contain the polymer matrix and at most one additive.
            - The composition for each dispens

In [4]:
###############################################################################
D, M, A = SetupDispenser("data/dispenser_setup.csv", "PLA")
###############################################################################

Dispenser 4: 100.0% of PLA, 
Dispenser 5: 90.0% of PLA, 10.0% of GNP, 
Dispenser 6: 90.0% of PLA, 10.0% of BN, 


Next, we read the file that contains the compositions using the Compositions function. It's working is detailed in the documentation:

In [5]:
help(Compositions)

Help on function Compositions in module DispenserSetup:

Compositions(compositions_file, dispensers_map, M, A, Polymer)
    Funtion to read the compositions file and define what compositions to dispense.
    
    Inputs:
    
        compositions_file: full route of a .csv file with the ddesired compositions. Example:
            Composition,PLA,GNP,CLO,BN
            1,0.95,0.02,0.0,0.03
            2,0.95,0.04,0.0,0.01
            
        dispensers_map, M, A: arrays returned from SetupDispenser function. 
        
    Outputs:
        
        C: list of arrays with the quantities to dispense from each dispenser for every composition in the file.



In [18]:
###############################################################################
compositions = Compositions("./data/compositions_to_dispense.csv", D, M, A, "PLA")
###############################################################################
print(C)

Composition  0 : Concentrations add up.
Composition  1 : Concentrations add up.
[array([0. , 0. , 0. , 0.5, 0.2, 0.3, 0. , 0. ]), array([0. , 0. , 0. , 0.5, 0.4, 0.1, 0. , 0. ])]


For the rest of the notebook we will use only the first of the compositions available in the .csv file. in your application you can easily loop over all of them, just remember to change the cup in the balance between compositions.

In [19]:
C = compositions[0]

It is convenient now to establish a ratio between materials: in this way, instead of dispensing all the ammount of one material and then of another, we can alternate small quantities and thus have a better mixing. We do this by selecting the one with lower ammount and expressing the others as a ratio with respect to this one. In every step we will dispense 1 g of the smaller one and the corresponding grams of the rest.

In [20]:
tol = 1e-3
minC = np.min([c for c in C if c>tol])
ratios = concentrations/minC 
print('The ratios for pellet dispensing will be: ', ratios)

The ratios for pellet dispensing will be:  [0.  0.  0.  2.5 1.  1.5 0.  0. ]


The next cell stablish commands to tare, calibrate and measure weight from the balance. You may need to edit them if your balance uses a different protocol.

In [ ]:
###############################################################################
def tare(balance):
    # Set the balance measurement to 0.0 g
    try:
        balance.write(b'T\r\n')
    except:
        balance = serial.Serial(port=balance_port, baudrate=9600, bytesize=serial.SEVENBITS,
                       parity=serial.PARITY_EVEN, stopbits=serial.STOPBITS_ONE)
        balance.write(b'T\r\n')


def calib(balance):
    # Calibrate the balance automatically
    try:
        balance.write(b'C\r\n')
    except:
        balance = serial.Serial(port=balance_port, baudrate=9600, bytesize=serial.SEVENBITS,
                       parity=serial.PARITY_EVEN, stopbits=serial.STOPBITS_ONE)
        balance.write(b'C\r\n')

def measure(balance):
    try:
        trash = balance.read_all()
        balance.write(b'B\r\n')
        reading = balance.readline()
        weight = float(reading[:10])
    except:
        balance = serial.Serial(port=balance_port, baudrate=9600, bytesize=serial.SEVENBITS,
                       parity=serial.PARITY_EVEN, stopbits=serial.STOPBITS_ONE)
        trash = balance.read_all()
        balance.write(b'B\r\n')
        reading = balance.readline()
        weight = float(reading[:10])
    return weight
###############################################################################

Select the total ammount of material that you want to dispense

In [28]:
###############################################################################
mass_target = 100 # g
###############################################################################

The next cell runs the dispensers until the desired quantity is reached. As pointed above, it will alternate dispensers keeping the ratios between materials.

In [ ]:
measureAll = True
massAll = 0
mass = 0
masses = [[] for _ in range(8)]
i = 0
times = [[] for _ in range(8)]
start = time.time()

dispenser_switch = {0 : b'0', 1 : b'1', 2 : b'2', 3 : b'3', 
                    4 : b'4', 5 : b'5', 6 : b'6', 7 : b'7', 'stop' : b'8'}



while measureAll == True:
    for j in range(len(C)):
        if C[j] > tol:
            if masses[j][i] < ratios[j]*(i+1):
                measureNow = True
                tare(balance)
                time.sleep(2)
                arduino.write(dispenser_switch[j])
                while measureNow == True:
                    mass = measure(balance)
                    if mass + masses[j][i] >= ratios[j]*(i+1):
                        arduino.write(dispenser_switch['stop'])
                        time.sleep(4)
                        mass = measure(balance)
                        measureNow = False
                massAll = massAll + mass
                masses[j].append(masses[j][i]+mass)
                time.sleep(0.1)
            elif masses[j][i] >= ratios[j]*(i+1):
                arduino.write(dispenser_switch['stop'])
            times[j].append(time.time()-start)
            print("iteration ", i+1, "| t = ", times[j][i+1], "s | ", "mass : ", masses[j][i+1])                  
    if massAll > mass_target:
        measureAll = False
        arduino.write(dispenser_switch['stop'])
    i = i + 1
    
arduino.write(dispenser_switch['stop'])

In [ ]:
import matplotlib.pyplot as plt

for key in D:
    plt.plot(time[D[key],:], mass[D[key],:],label=D)
plt.title("Progression of Sample")
plt.xlabel("Time [s]")
plt.ylabel("Mass [g]")
plt.legend()
plt.show()

In [ ]:
massAll

In [ ]:
error = X[1]abs((mass4[i] * 0.1)/(mass4[i] + mass1[i]))/(massAll)*100
accuracy = 100 - error
print('Accuracy:', '%.2f' % accuracy,'%')

rate = massAll/finish4[i]
rate = rate * 3600
print('Mass flow rate: ', '%.2f' % rate,'g/h')

In [17]:
79*'#'

'###############################################################################'

In [23]:
for key in D:
    print(key)

PLA
GNP
BN


In [30]:
np.sum(abs(masses[:,-1]-C*mass_target)/np.sum(C*mass_target)

array([ 0.,  0.,  0., 50., 20., 30.,  0.,  0.])